# Image Colorization

In [ ]:
# Project Outline

# 1 Imports
# 2 Data
# 3 Preprocessing 
# 4 Model 
# 5 Training
# 6 Evaluation
# 7 Visualization

In [ ]:
# Imports
import numpy as np

In [ ]:
# Data
l_path = "/Data/Raw/gray_scale.npy"
ab_paths = ["/Data/Raw/ab/ab1.npy", "/Data/Raw/ab/ab2.npy", "/Data/Raw/ab/ab3.npy"]

l_data = np.load(l_path)
ab_list = [np.load(path) for path in ab_paths]
ab_data = np.concatenate(ab_list, axis=0)